In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ETL pipeline

Get covid cases data in germany. 
Data is taken from here: https://www.arcgis.com/home/item.html?id=f10774f1c63e40168479a1feb6c7ca74

In [3]:
df_rki = pd.read_csv('https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data')
#df_rki = pd.read_csv('data/RKI_COVID19-2.csv')
df_rki['Meldedatum'] = pd.to_datetime(df_rki['Meldedatum'], format='%Y/%m/%d')

## Stats overview

In [4]:
n_cases = df_rki.loc[df_rki['NeuerFall'].isin([0,1])]['AnzahlFall'].sum()
n_cases_new = df_rki.loc[df_rki['NeuerFall'].isin([-1,1])]['AnzahlFall'].sum()
n_deaths = df_rki.loc[df_rki['NeuerTodesfall'].isin([0,1])]['AnzahlTodesfall'].sum()
n_deaths_new = df_rki.loc[df_rki['NeuerTodesfall'].isin([-1,1])]['AnzahlTodesfall'].sum()
n_recovered = df_rki.loc[df_rki['NeuGenesen'].isin([0,1])]['AnzahlGenesen'].sum()
n_recovered_new = df_rki.loc[df_rki['NeuGenesen'].isin([-1,1])]['AnzahlGenesen'].sum()
n_active = n_cases - n_deaths - n_recovered
n_active_new = n_cases_new - n_deaths_new - n_recovered_new

df_stats = pd.DataFrame({' ':['Total','Today'],\
                    'Cases':[n_cases, n_cases_new],\
                    'Recovered':[n_recovered, n_recovered_new],\
                    'Deaths':[n_deaths, n_deaths_new],\
                    'Active':[n_active, n_active_new]})
df_stats.set_index(' ', inplace=True)
n_cases_new

797

In [5]:
df_cases = df_rki.loc[df_rki['NeuerFall'].isin([0,1])].copy()
df_deaths = df_rki.loc[df_rki['NeuerTodesfall'].isin([0,1])].copy()
df_recovered = df_rki.loc[df_rki['NeuGenesen'].isin([0,1])].copy()

In [6]:
df_deaths_stats = pd.get_dummies(df_deaths['Altersgruppe']).sum().reset_index()
df_deaths_stats.rename(columns={'index':'Age',0:'Count'}, inplace=True)
df_deaths_stats

,Age,Count
0,A00-A04,1
1,A15-A34,16
2,A35-A59,344
3,A60-A79,2500
4,A80+,4666


In [7]:
df_cases.drop(columns=['AnzahlTodesfall','NeuerTodesfall','AnzahlGenesen','NeuGenesen','Altersgruppe2'], inplace=True)
df_deaths.drop(columns=['AnzahlFall','NeuerFall','NeuGenesen','AnzahlGenesen','Altersgruppe2'], inplace=True)
df_recovered.drop(columns=['AnzahlFall','NeuerFall','AnzahlTodesfall','NeuerTodesfall','Altersgruppe2'], inplace=True)

Data cleaining:
- We'll later add population data to all districts. The data we pull is for all districts expect of the quaters of Berlin. We thus merge the Berlin data to view it as a single city.
- Converting the dates to proper datetime data types
- drop column `Altersgruppe2` since this is not published any more.
- drop column `Datenstand` as this is always the current date

In [8]:
# merge Berlin cases since we currently don't have population data for the individual districts
df_cases.loc[df_cases['IdLandkreis'].isin(np.arange(11000,11013,1)),'IdLandkreis'] = 11000
df_cases.loc[df_cases['IdLandkreis'].isin(np.arange(11000,11013,1)),'Landkreis'] = 'SK Berlin'
# convert columns to datetimes
#df_cases['Meldedatum'] = df_cases['Meldedatum'].apply(pd.to_datetime)
#df_cases['Datenstand'] = df_cases['Datenstand'].apply(lambda x: pd.to_datetime(x.split(',')[0]))
#df_cases['Refdatum'] = df_cases['Refdatum'].apply(pd.to_datetime)
# drop irrelevant columns
df_cases.drop(columns = ['Datenstand'], inplace=True)
df_cases.head()

,FID,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,Meldedatum,IdLandkreis,NeuerFall,Refdatum,IstErkrankungsbeginn
0,7789333,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-14,1001,0,2020/03/16 00:00:00,1
1,7789334,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-19,1001,0,2020/03/13 00:00:00,1
2,7789335,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-19,1001,0,2020/03/16 00:00:00,1
3,7789336,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-21,1001,0,2020/03/13 00:00:00,1
4,7789337,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-27,1001,0,2020/03/22 00:00:00,1


In [9]:
# one-hot encode age groups
df_cases = pd.concat([df_cases,pd.get_dummies(df_cases['Altersgruppe'], prefix='age')],axis=1).drop(columns='Altersgruppe')
df_cases.head()

,FID,IdBundesland,Bundesland,Landkreis,Geschlecht,AnzahlFall,Meldedatum,IdLandkreis,NeuerFall,Refdatum,IstErkrankungsbeginn,age_A00-A04,age_A05-A14,age_A15-A34,age_A35-A59,age_A60-A79,age_A80+,age_unbekannt
0,7789333,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-14,1001,0,2020/03/16 00:00:00,1,0,0,1,0,0,0,0
1,7789334,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-19,1001,0,2020/03/13 00:00:00,1,0,0,1,0,0,0,0
2,7789335,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-19,1001,0,2020/03/16 00:00:00,1,0,0,1,0,0,0,0
3,7789336,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-21,1001,0,2020/03/13 00:00:00,1,0,0,1,0,0,0,0
4,7789337,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-27,1001,0,2020/03/22 00:00:00,1,0,0,1,0,0,0,0


Next, we pull the population data

Data on inhabitants:
https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html
could be correct one

In [10]:
# needs packae xlrd
df_population = pd.read_excel('./data/04-kreise.xlsx', sheet_name='Kreisfreie Städte u. Landkreise',skiprows=6, skipfooter=16)
df_population.columns = ['IdLandkreis', 'Bezeichnung','Name','NUTS3','area','pop_tot','pop_male','pop_female','pop_per_sqkm2']
df_population.dropna(axis=0, how='any', inplace=True)
# set integer IdLandkreis
df_population['IdLandkreis'] = df_population['IdLandkreis'].astype(int)
df_population.head(10)

,IdLandkreis,Bezeichnung,Name,NUTS3,area,pop_tot,pop_male,pop_female,pop_per_sqkm2
1,1001,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0
2,1002,Kreisfreie Stadt,"Kiel, Landeshauptstadt",DEF02,118.65,247548.0,120566.0,126982.0,2086.0
3,1003,Kreisfreie Stadt,"Lübeck, Hansestadt",DEF03,214.19,217198.0,104371.0,112827.0,1014.0
4,1004,Kreisfreie Stadt,"Neumünster, Stadt",DEF04,71.66,79487.0,39241.0,40246.0,1109.0
5,1051,Kreis,Dithmarschen,DEF05,1428.17,133210.0,65720.0,67490.0,93.0
6,1053,Kreis,Herzogtum Lauenburg,DEF06,1263.07,197264.0,96881.0,100383.0,156.0
7,1054,Kreis,Nordfriesland,DEF07,2083.55,165507.0,81099.0,84408.0,79.0
8,1055,Kreis,Ostholstein,DEF08,1393.02,200581.0,96765.0,103816.0,144.0
9,1056,Kreis,Pinneberg,DEF09,664.25,314391.0,154211.0,160180.0,473.0
10,1057,Kreis,Plön,DEF0A,1083.56,128647.0,62532.0,66115.0,119.0


# Rolling

Create a data frame that has the rolling sum of the cases in the past week

In [17]:
df_cases.copy()[['Meldedatum','AnzahlFall','IdLandkreis']].groupby(['Meldedatum','IdLandkreis']).sum().reset_index()

,Meldedatum,IdLandkreis,Landkreis,AnzahlFall
0,2020-01-28,5513,SK Gelsenkirchen,2
1,2020-01-28,9181,LK Landsberg a.Lech,1
2,2020-01-28,9188,LK Starnberg,1
3,2020-01-29,9162,SK München,1
4,2020-01-29,9179,LK Fürstenfeldbruck,1
...,...,...,...,...
20204,2020-05-19,14628,LK Sächsische Schweiz-Osterzgebirge,4
20205,2020-05-19,16066,LK Schmalkalden-Meiningen,1
20206,2020-05-19,16069,LK Hildburghausen,1
20207,2020-05-19,16075,LK Saale-Orla-Kreis,3


In [18]:
#df_cases.groupby('IdLandkreis').rolling('7d').sum()
df_to_roll = df_cases.copy()[['Meldedatum','AnzahlFall','IdLandkreis','Landkreis']].groupby(['Meldedatum','IdLandkreis','Landkreis']).sum().reset_index()
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll


,Meldedatum,IdLandkreis,Landkreis,AnzahlFall
0,2020-01-28,5513,SK Gelsenkirchen,2
1,2020-01-28,9181,LK Landsberg a.Lech,1
2,2020-01-28,9188,LK Starnberg,1
3,2020-01-29,9162,SK München,1
4,2020-01-29,9179,LK Fürstenfeldbruck,1
...,...,...,...,...
20124,2020-05-19,5762,LK Höxter,2
20123,2020-05-19,5754,LK Gütersloh,3
20122,2020-05-19,5711,SK Bielefeld,1
20134,2020-05-19,6433,LK Groß-Gerau,3


In [ ]:
#lkr_all - set(df_to_roll.loc[df_to_roll['Meldedatum'] == date ]['IdLandkreis'])

In [ ]:
#pd.date_range(df_to_roll['Meldedatum'].min(), df_to_roll['Meldedatum'].max(), freq = '1D')

To compute rolling 7-day totals of new cases, we need to make sure to have one row per day for every administrative district. Thus, we to add rows with zero cases for all days that are missing.

In [19]:
lkr_all = set(df_to_roll['IdLandkreis'].unique())
# sweep over all days in the data set
for date in pd.date_range(df_to_roll['Meldedatum'].min(), df_to_roll['Meldedatum'].max(), freq = '1D'):
    # add zero rows for all districts that didn't report cases on that day
    for id_lkr in lkr_all - set(df_to_roll.loc[df_to_roll['Meldedatum'] == date ]['IdLandkreis']):
        df_to_roll = df_to_roll.append({'Meldedatum':date, 'IdLandkreis': id_lkr, 'AnzahlFall':0}, ignore_index=True)
    print('fixed date',date)        

fixed date 2020-01-28 00:00:00
fixed date 2020-01-29 00:00:00
fixed date 2020-01-30 00:00:00
fixed date 2020-01-31 00:00:00
fixed date 2020-02-01 00:00:00
fixed date 2020-02-02 00:00:00
fixed date 2020-02-03 00:00:00
fixed date 2020-02-04 00:00:00
fixed date 2020-02-05 00:00:00
fixed date 2020-02-06 00:00:00
fixed date 2020-02-07 00:00:00
fixed date 2020-02-08 00:00:00
fixed date 2020-02-09 00:00:00
fixed date 2020-02-10 00:00:00
fixed date 2020-02-11 00:00:00
fixed date 2020-02-12 00:00:00
fixed date 2020-02-13 00:00:00
fixed date 2020-02-14 00:00:00
fixed date 2020-02-15 00:00:00
fixed date 2020-02-16 00:00:00
fixed date 2020-02-17 00:00:00
fixed date 2020-02-18 00:00:00
fixed date 2020-02-19 00:00:00
fixed date 2020-02-20 00:00:00
fixed date 2020-02-21 00:00:00
fixed date 2020-02-22 00:00:00
fixed date 2020-02-23 00:00:00
fixed date 2020-02-24 00:00:00
fixed date 2020-02-25 00:00:00
fixed date 2020-02-26 00:00:00
fixed date 2020-02-27 00:00:00
fixed date 2020-02-28 00:00:00
fixed da

In [21]:
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll

,Meldedatum,IdLandkreis,Landkreis,AnzahlFall
0,2020-01-28,5513,SK Gelsenkirchen,2
20233,2020-01-28,8237,NaN,0
20258,2020-01-28,3151,NaN,0
20232,2020-01-28,8236,NaN,0
20211,2020-01-28,5124,NaN,0
...,...,...,...,...
45293,2020-05-19,7135,NaN,0
45294,2020-05-19,7137,NaN,0
45295,2020-05-19,9183,NaN,0
45285,2020-05-19,9174,NaN,0


In [22]:
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll = df_to_roll.set_index('Meldedatum').groupby('IdLandkreis').rolling('7d').sum()

df_to_roll = df_to_roll.drop(columns = ['IdLandkreis']).reset_index()
#df_to_roll.loc[df_to_roll['IdLandkreis'] == 5558]

In [ ]:
df_cases_roll = pd.merge(df_to_roll, df_population,on='IdLandkreis')
df_cases_roll.insert(3,'AnzahlFall100k',0)
df_cases_roll['AnzahlFall100k'] = df_cases_roll['AnzahlFall']/df_cases_roll['pop_tot']*(10**5)
df_cases_roll
df_cases_roll.loc[df_cases_roll['IdLandkreis']==5558].tail(25)

In [ ]:
df_cases_roll['Landkreis'] = df_cases_roll['IdLandkreis']
# this step is a bit slow, could likely be improved
df_cases_roll['Landkreis'] = df_cases_roll['Landkreis'].apply(lambda x: df_cases.loc[df_cases['IdLandkreis'] == x]['Landkreis'].iloc[0])

In [ ]:
df_cases_roll
df_cases_roll.rename(columns={"AnzahlFall":"7d_AnzahlFall",'AnzahlFall100k':'7d_AnzahlFall100k'})

# Cases overview

In [ ]:
df_rki.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall','AnzahlTodesfall']]

# Long data format

In [ ]:
df_ctr_cases = df_cases.groupby(['Meldedatum'])\
                        .sum()[['AnzahlFall']]
df_ctr_deaths = df_deaths.groupby(['Meldedatum'])\
                        .sum()[['AnzahlTodesfall']]
df_ctr_recovered = df_recovered.groupby(['Meldedatum'])\
                        .sum()[['AnzahlGenesen']]
df_ctr_cases = pd.melt(df_ctr_cases.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlFall'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr_deaths = pd.melt(df_ctr_deaths.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlTodesfall'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr_recovered = pd.melt(df_ctr_recovered.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlGenesen'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr = pd.concat([df_ctr_cases,df_ctr_deaths,df_ctr_recovered], axis = 0)
df_ctr['category'] = df_ctr['category']\
        .apply(lambda x: 'case' if x == 'AnzahlFall' else\
                              ('death' if x == 'AnzahlTodesfall' else 'recovered'))
df_ctr

In [ ]:
df_ctr_cum = df_ctr.copy().sort_values(by=['Meldedatum','category'])
for el in list(df_ctr_cum['category'].unique()):
    df_ctr_cum.loc[df_ctr_cum['category']== el,'Number' ] = \
        np.cumsum(df_ctr_cum.loc[df_ctr_cum['category']== el,'Number' ])
df_ctr_cum    

### States

In [ ]:
df_sta = pd.concat([df_cases.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall']],\
            df_deaths.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                            [['Meldedatum','Bundesland','AnzahlTodesfall']],\
            df_recovered.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                            [['Meldedatum','Bundesland','AnzahlGenesen']]])
df_sta = df_sta.fillna(0).groupby(['Meldedatum','Bundesland']).sum().reset_index()
df_sta[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']] = df_sta[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']].astype('int64')
df_sta

In [ ]:
df_sta_cum = df_sta.copy()
for state in list(df_sta['Bundesland'].unique()):
    for col in ['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']:
        df_sta_cum.loc[df_sta_cum['Bundesland']==state,col] = np.cumsum(df_sta_cum.loc[df_sta_cum['Bundesland']==state,col])
df_sta_cum

### Districts

In [ ]:
df_recovered.groupby(['Meldedatum','Landkreis']).sum().reset_index()

In [ ]:
df_lkr = pd.concat([df_cases.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                [['Meldedatum','Landkreis','AnzahlFall']],\
            df_deaths.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                            [['Meldedatum','Landkreis','AnzahlTodesfall']],\
            df_recovered.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                            [['Meldedatum','Landkreis','AnzahlGenesen']]])
df_lkr = df_lkr.fillna(0).groupby(['Meldedatum','Landkreis']).sum().reset_index()
df_lkr[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']] = df_lkr[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']].astype('int64')
df_lkr

In [ ]:
df_lkr_cum = df_lkr.copy()
for el in list(df_lkr['Landkreis'].unique()):
    for col in ['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']:
        df_lkr_cum.loc[df_lkr_cum['Landkreis']==el,col] = np.cumsum(df_lkr_cum.loc[df_lkr_cum['Landkreis']==el,col])
df_lkr_cum

Location Data

Collected the coordinates of the individual districts on this webpage:
https://public.opendatasoft.com/explore/dataset/landkreise-in-germany/export/

In [ ]:
pd.read_csv?

In [ ]:
#geo_data = pd.read_csv('https://public.opendatasoft.com/explore/dataset/landkreise-in-germany/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B')
geo_data = pd.read_csv('landkreise-in-germany.csv', delimiter = ';',\
                       usecols=['Geo Point','Name 2','Cca 2', 'Type 2'])
# drop NaN row corresponding to a lake
geo_data.dropna(axis=0, inplace = True)
geo_data[['lat','lon']] = geo_data['Geo Point'].str.split(',', expand=True)
geo_data.drop(columns = 'Geo Point',inplace = True)

In [ ]:
geo_data.rename(columns = {'Name 2':'Name','Cca 2':'IdLandkreis','Type 2':'Type of District'}, inplace =True)
geo_data

In [ ]:
df_cases_loc = pd.merge(df_cases, geo_data, on='IdLandkreis')[['IdLandkreis','Meldedatum','AnzahlFall','lat','lon']]
df_cases_loc

When plotting the map, we only use the geological coordinates of the reported case. Currently, each row contains information about how many cases were reported. Thus, we create a new row for every reported cases and copy the coordinates of the district.

In [ ]:
df_cases_loc_long = df_cases_loc.loc[df_cases_loc['AnzahlFall'] == 1]
for n_cases in sorted(df_cases_loc['AnzahlFall'].unique())[1:]:
    for k in range(n_cases):
        df_cases_loc_long = \
            pd.concat([df_cases_loc_long, df_cases_loc.loc[df_cases_loc['AnzahlFall'] == n_cases]])
df_cases_loc_long.drop(columns='AnzahlFall', inplace = True)

In [ ]:
df_cases_loc_long['Meldedatum']

In [ ]:
(pd.Timestamp.today() - df_cases_loc_long['Meldedatum'].head()).dt.days < 7

In [ ]:
df_cases_7d = df_cases_loc_long.loc[(pd.Timestamp.today() - df_cases_loc_long['Meldedatum']).dt.days < 7]

In [ ]:
df_cases_7d

## Save Data

In [ ]:
#df_cases.to_csv('data_cases.csv', index=False)
#df_deaths.to_csv('data_deaths.csv', index=False)
#df_recovered.to_csv('data_recovered.csv', index=False)
df_deaths_stats.to_csv('data_death_stats.csv', index=False)
df_stats.to_csv('data_stats.csv')
df_cases_roll.to_csv('data_cases_rolling.csv', index=False)
#df_population.to_csv('data_population.csv', index=False)
# country data
df_ctr.to_csv('data_ctr_long.csv', index=False)
df_ctr_cum.to_csv('data_ctr_cum_long.csv', index=False)

In [ ]:
# state data
df_sta.to_csv('data_sta_long.csv', index=False)
df_sta_cum.to_csv('data_sta_cum_long.csv', index=False)

In [ ]:
# district data
df_lkr.to_csv('data_lkr_long.csv', index=False)
df_lkr_cum.to_csv('data_lkr_cum_long.csv', index=False)

In [ ]:
# location case data 
df_cases_loc_long.to_csv('data_loc_long.csv', index=False)